In [1]:
import cv2 #For computer vision tasks
import pandas as pd #For data manipulation and analysis
from ultralytics import YOLO #For object detection model
import cvzone #Utility functions for computer vision tasks
import numpy as np #For numerical operations
import pytesseract #For Optical Character Recognition Images data convert to Text data
from datetime import datetime #Handling dates and times in saved text 

In [2]:
pytesseract.pytesseract.tesseract_cmd = r"C:/Program Files/Tesseract-OCR/tesseract.exe" #For calling pytesseract

model = YOLO("C:/Users/ikhwa/Desktop/PLATE NUMBER DETECTION/carnumberplate-main/carnumberplate-main/best.pt") #For calling YOLO trained model to detecting plate numbers

In [3]:
def RGB(event, x, y, flags, param): # Mouse Event Callback function to print the coordinates of the mouse pointer or axis
    if event == cv2.EVENT_MOUSEMOVE:
        point = [x, y] #To coordinates the axis of boundaring box
        print(point)

In [ ]:
#cap = cv2.VideoCapture('JLNGOMBAK.mp4') #Initialized video capture
for i in range(1):#The loops will run once with 'i' being '0'
    cap = cv2.VideoCapture(i) #To open the video capture(webcam) using OpenCV
    if cap.isOpened(): #To check video capture video successfully opened
        print(f"Webcam found at index {i}") #If the webcam found,print found
        break #Exit the loop after the webcam was found


my_file = open("coco1.txt", "r") #To read and class plate number trained data
data = my_file.read() #To read contents file
class_list = data.split("\n") #To splits the data into list of class labels

area = [(35, 378), (16, 456), (1015, 451), (965, 378)] #Point of axis to place Region Of Interest

count = 0 #Keeps track of frames processed
list1 = [] #LIst to store detected plate numbers
processed_numbers = set() #Set to avoid processing the same plate multiple times

In [ ]:
#Main Processing Loop
with open("car_plate_data.txt", "a") as file: 
    file.write("NumberPlate\tDate\tTime\n")  # Writing column headers for saved text


   
    frame = cv2.resize(frame, (1020, 500)) #Resizes the frame for consistency
    results = model.predict(frame) #Uses the YOLO model to detect plate numbers in the frame
    a = results[0].boxes.cpu().data  #Extracts the detection results
    px = pd.DataFrame(a).astype("float") #Converts detection results to a pandas DataFrame

   
    for index, row in px.iterrows(): #To iterates thru each detected plate numbers
        x1 = int(row[0]) #Coordinates of the bounding box around the detected object
        y1 = int(row[1]) #Coordinates of the bounding box around the detected object
        x2 = int(row[2]) #Coordinates of the bounding box around the detected object
        y2 = int(row[3]) #Coordinates of the bounding box around the detected object
        
        d = int(row[5]) #Class ID of the detected object
        c = class_list[d] #Class label to class ID
        cx = int(x1 + x2) // 2 #Coordinates of the centre bounding box
        cy = int(y1 + y2) // 2 #Coordinates of the centre bounding box
        result = cv2.pointPolygonTest(np.array(area, np.int32), ((cx, cy)), False) #To Checks if the center point is inside the defined polygonal area
        if result >= 0:
           crop = frame[y1:y2, x1:x2] #Crops the detected plate region from the window frame
           gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY) #To converts the cropped image to Grayscale
           gray = cv2.bilateralFilter(gray, 10, 20, 20) #To applied filter to reduce noise

           text = pytesseract.image_to_string(gray).strip() #Uses OCR to extract text from the processed image
           text = text.replace('(', '').replace(')', '').replace(',', '').replace(']','') #To cleans up the extracted text
           if text not in processed_numbers:
              processed_numbers.add(text) #to adds the detected plate to set of processed numbers
              list1.append(text) #To adds the detected plate to the list
              current_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S") #To gets the current date and time
              with open("car_plate_data.txt", "a") as file:
                   file.write(f"{text}\t{current_datetime}\n") #To writes the detected plate and timestamp to the file
                   cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2) #To draws a rectangle around the detected plate
                   cv2.imshow('crop', crop) #To displays the cropped image

      
    cv2.polylines(frame, [np.array(area, np.int32)], True, (255, 0, 0), 2) #To draws the polygonal area on the frame
    cv2.imshow("RGB", frame) #To displays the processed frame
    #if cv2.waitKey(1) & 0xFF ==27: #For stop video run per framing
    if cv2.waitKey(1) & 0xFF == ord('q'): #To set the key to exit the loop
        break

cap.release()   #To releases the video or webcam 
cv2.destroyAllWindows() #To closes all OpenCV windows


0: 320x640 1 number plate, 46.9ms
Speed: 6.1ms preprocess, 46.9ms inference, 8.7ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 number plate, 41.9ms
Speed: 5.3ms preprocess, 41.9ms inference, 6.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 number plate, 39.5ms
Speed: 4.0ms preprocess, 39.5ms inference, 4.9ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 number plate, 37.9ms
Speed: 9.5ms preprocess, 37.9ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 number plate, 40.0ms
Speed: 0.0ms preprocess, 40.0ms inference, 15.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 number plate, 34.7ms
Speed: 4.1ms preprocess, 34.7ms inference, 3.8ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 number plate, 34.5ms
Speed: 3.0ms preprocess, 34.5ms inference, 3.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 number plate, 30.4ms
Speed: 3.0ms preprocess, 30.4ms inferenc